### Главы 13 - 14

Тензоры

In [1]:
import numpy as np

class Tensor(object):
    
    def __init__(self, data):
        self.data = np.array(data)
        
    def __add__(self, other):
        return Tensor(self.data + other.data)
    
    def __repr__(self):
        return str(self.data.__repr__())
    
    def __str__(self):
        return str(self.data.__str__())

In [2]:
x = Tensor([1,2,3,4,5])
print(x)

[1 2 3 4 5]


In [3]:
y = x + x
print(y)

[ 2  4  6  8 10]


In [4]:
print(repr(y))

array([ 2,  4,  6,  8, 10])


In [5]:
print(str(y))

[ 2  4  6  8 10]


In [6]:
import numpy as np

class Tensor(object):
    
    def __init__(self, data, creators=None, creation_op=None):
        self.data = np.array(data)
        self.creators = creators # список любых тензоров, используемых для создания текущего тензора
        self.creation_op = creation_op # хранит операции, испольщзуемые для создания текущего тензора
        self.grad = None
        
    def backward(self, grad):
        """Рекурсивно распространяет назад градиент
        от потомков к родителям """
        self.grad = grad
        
        if self.creation_op == "add":
            self.creators[0].backward(grad)
            self.creators[1].backward(grad)
        
    def __add__(self, other):
        "Создает вычислительный граф"
        return Tensor(self.data + other.data, 
                     creators = [self, other],
                     creation_op = "add")
    
    def __repr__(self):
        return str(self.data.__repr__())
    
    def __str__(self):
        return str(self.data.__str__())

In [7]:
x = Tensor([1,2,3,4,5])
y = Tensor([2,2,2,2,2])
z = x + y
z.backward(Tensor(np.array([1,1,1,1,1])))
print(x.grad)
print(y.grad)
print(z.creators)
print(z.creation_op)

[1 1 1 1 1]
[1 1 1 1 1]
[array([1, 2, 3, 4, 5]), array([2, 2, 2, 2, 2])]
add


In [8]:
a = Tensor([1,2,3,4,5])
b = Tensor([2,2,2,2,2])
c = Tensor([5,4,3,2,1])
d = Tensor([-1,-2,-3,-4,-5])

e = a + b
f = c + d
g  = e + f

g.backward(Tensor(np.array([1,1,1,1,1])))
print(a.grad)

[1 1 1 1 1]


Устраним ошибку распространения. В узел, дважды задействованном в создании потомков градиент должен вернуться дважды: [2,2,2,2,2]

In [9]:
a = Tensor([1,2,3,4,5])
b = Tensor([2,2,2,2,2])
c = Tensor([5,4,3,2,1])

d = a + b
e = b + c
f = d + e

f.backward(Tensor(np.array([1,1,1,1,1])))
print(b.grad.data == np.array([2,2,2,2,2]))

[False False False False False]


In [22]:
import numpy as np

class Tensor(object):
        
    def __init__(self, data, # вектор (список)
                 autograd = False, 
                 creators=None, 
                 creation_op=None, 
                 id_ = None):
        self.data = np.array(data) # принимаем значение атрибута в виде массива numpy
        self.creators = creators # список любых тензоров, используемых для создания текущего тензора
        self.creation_op = creation_op # хранит операции, используемые для создания текущего тензора
        self.grad = None # вначале градиента нет
        self.autograd = autograd # включатель автоградиента (нужно ли для конкретного нейрона вычислять градиент)
        self.children = {} # счетчик для учета количества градиентов, полученных от каждого потомка при обратном распространении
        
        if id_ is None: # если при инициировании инстанса нет айди, создать случайный
            self.id_ = np.random.randint(0, 100_000)
        else: # если при инициировании инстанса указан айди, присвоить его 
            self.id_ = id_
        
        if creators is not None: # если есть родители
            for c in creators: # для каждого родителя
                if self.id_ not in c.children: # если айди инстанса не среди айди потомков выбранного родителя
                    c.children[self.id_] = 1 # записать его в потомки выбранного родителя
                else: # если айди уже среди айди потомков
                    c.children[self.id_] += 1 # добавить еще единичку (т.е. учесть, через какое количество веток 
                                                            # наш инстанс является наследником)
    
    def all_children_grads_accounted_for(self):
        """Проверяет, получил ли тензор градиенты от всех своих потомков в графе"""
        for id_, cnt in self.children.items(): # словарь self.children имеет ключи id_ и значения 1,2,...
            if cnt != 0: # если наследников нет, вернуть False
                return False
        return True # если наследник(и) есть, вернуть True
    
    def backward(self,grad=None, grad_origin=None):
        """Рекурсивно распространяет назад градиент
        от потомков к родителям """
        if(self.autograd): # если включатель автоградиента вкл. (если для нейрона нужно вычислять градиент)
#             if(grad is None): # если градиента пока никакого нет
#                 grad = FloatTensor(np.ones_like(self.data))
            
            if(grad_origin is not None): # если есть градиент
                if(self.children[grad_origin.id_] == 0): # и если потомков дальше нет
                    raise Exception("cannot backprop more than once") # не распространять градиент
                else: # если потомки есть
                    self.children[grad_origin.id_] -= 1 # скинуть счетчик потомков на 1

            if(self.grad is None): # если у инстанса нет градиента
                self.grad = grad # присвоить атрибут метода grad
            else: # если есть градиент
                self.grad += grad # добавить к нему значение атрибута метода grad
            
            assert grad.autograd == False # проверка: если для градиента включен автоградиент, программу завалить
            
            # only continue backpropping if there's something to
            # backprop into and if all gradients (from children)
            # are accounted for override waiting for children if
            # "backprop" was called on this variable directly
            if(self.creators is not None and # если есть родители и одновременно есть наследники или grad_origin is None
               (self.all_children_grads_accounted_for() or 
                grad_origin is None)):

                if(self.creation_op == "add"): # если операция сложения
                    self.creators[0].backward(self.grad, self) # распространяем градиент на папу
                    self.creators[1].backward(self.grad, self) # и на маму
    
    def __add__(self, other):
        "Создает вычислительный граф"
        if self.autograd and other.autograd:
            
            return Tensor(self.data + other.data,
                            autograd=True,
                            creators = [self, other],
                            creation_op = "add")
        return Tensor(self.data + other.data)
    
    def __repr__(self):
        return str(self.data.__repr__())
    
    def __str__(self):
        return str(self.data.__str__())

In [23]:
a = Tensor([1,2,3,4,5], autograd=True)
b = Tensor([2,2,2,2,2], autograd=True)
c = Tensor([5,4,3,2,1], autograd=True)

d = a + b
e = b + c
f = d + e

f.backward(Tensor(np.array([1,1,1,1,1])))
print(b.grad.data == np.array([2,2,2,2,2]))

[ True  True  True  True  True]


Реализуем поддержку отрицания

In [24]:
import numpy as np

class Tensor(object):
        
    def __init__(self, data, # вектор (список)
                 autograd = False, 
                 creators=None, 
                 creation_op=None, 
                 id_ = None):
        self.data = np.array(data) # принимаем значение атрибута в виде массива numpy
        self.creators = creators # список любых тензоров, используемых для создания текущего тензора
        self.creation_op = creation_op # хранит операции, используемые для создания текущего тензора
        self.grad = None # вначале градиента нет
        self.autograd = autograd # включатель автоградиента (нужно ли для конкретного нейрона вычислять градиент)
        self.children = {} # счетчик для учета количества градиентов, полученных от каждого потомка при обратном распространении
        
        if id_ is None: # если при инициировании инстанса нет айди, создать случайный
            self.id_ = np.random.randint(0, 100_000)
        else: # если при инициировании инстанса указан айди, присвоить его 
            self.id_ = id_
        
        if creators is not None: # если есть родители
            for c in creators: # для каждого родителя
                if self.id_ not in c.children: # если айди инстанса не среди айди потомков выбранного родителя
                    c.children[self.id_] = 1 # записать его в потомки выбранного родителя
                else: # если айди уже среди айди потомков
                    c.children[self.id_] += 1 # добавить еще единичку (т.е. учесть, через какое количество веток 
                                                            # наш инстанс является наследником)
    
    def all_children_grads_accounted_for(self):
        """Проверяет, получил ли тензор градиенты от всех своих потомков в графе"""
        for id_, cnt in self.children.items(): # словарь self.children имеет ключи id_ и значения 1,2,...
            if cnt != 0: # если наследников нет, вернуть False
                return False
        return True # если наследник(и) есть, вернуть True
    
    def backward(self,grad=None, grad_origin=None):
        """Рекурсивно распространяет назад градиент
        от потомков к родителям """
        if(self.autograd): # если включатель автоградиента вкл. (если для нейрона нужно вычислять градиент)
#             if(grad is None): # если градиента пока никакого нет
#                 grad = FloatTensor(np.ones_like(self.data))
            
            if(grad_origin is not None): # если есть градиент
                if(self.children[grad_origin.id_] == 0): # и если потомков дальше нет
                    raise Exception("cannot backprop more than once") # не распространять градиент
                else: # если потомки есть
                    self.children[grad_origin.id_] -= 1 # скинуть счетчик потомков на 1

            if(self.grad is None): # если у инстанса нет градиента
                self.grad = grad # присвоить атрибут метода grad
            else: # если есть градиент
                self.grad += grad # добавить к нему значение атрибута метода grad
            
            assert grad.autograd == False # проверка: если для градиента включен автоградиент, программу завалить
            
            # only continue backpropping if there's something to
            # backprop into and if all gradients (from children)
            # are accounted for override waiting for children if
            # "backprop" was called on this variable directly
            if(self.creators is not None and # если есть родители и одновременно есть наследники или grad_origin is None
               (self.all_children_grads_accounted_for() or 
                grad_origin is None)):

                if(self.creation_op == "add"): # если операция сложения
                    self.creators[0].backward(self.grad, self) # распространяем градиент на папу
                    self.creators[1].backward(self.grad, self) # и на маму
                if(self.creation_op == "neg"): # если операция сложения
                    self.creators[0].backward(self.grad.__neg__()) # распространяем градиент на папу
    
    def __add__(self, other):
        "Создает вычислительный граф"
        if self.autograd and other.autograd:
            
            return Tensor(self.data + other.data,
                            autograd=True,
                            creators = [self, other],
                            creation_op = "add")
        return Tensor(self.data + other.data)
    
    def __neg__(self):
        if self.autograd:
            return Tensor(self.data * (-1),
                 autograd = True, 
                 creators=[self], 
                 creation_op="neg")
        return Tensor(self.data * (-1))
    
    def __repr__(self):
        return str(self.data.__repr__())
    
    def __str__(self):
        return str(self.data.__str__())

In [25]:
a = Tensor([1,2,3,4,5], autograd=True)
b = Tensor([2,2,2,2,2], autograd=True)
c = Tensor([5,4,3,2,1], autograd=True)

d = a + (-b)
e = (-b) + c
f = d + e

f.backward(Tensor(np.array([1,1,1,1,1])))
print(b.grad.data == np.array([-2,-2,-2,-2,-2]))

[ True  True  True  True  True]


Добавление других операций

In [40]:
import numpy as np

class Tensor(object):
        
    def __init__(self, data, # вектор (список)
                 autograd = False, 
                 creators=None, 
                 creation_op=None, 
                 id_ = None):
        self.data = np.array(data) # принимаем значение атрибута в виде массива numpy
        self.creators = creators # список любых тензоров, используемых для создания текущего тензора
        self.creation_op = creation_op # хранит операции, используемые для создания текущего тензора
        self.grad = None # вначале градиента нет
        self.autograd = autograd # включатель автоградиента (нужно ли для конкретного нейрона вычислять градиент)
        self.children = {} # счетчик для учета количества градиентов, полученных от каждого потомка при обратном распространении
        
        if id_ is None: # если при инициировании инстанса нет айди, создать случайный
            self.id_ = np.random.randint(0, 100_000)
        else: # если при инициировании инстанса указан айди, присвоить его 
            self.id_ = id_
        
        if creators is not None: # если есть родители
            for c in creators: # для каждого родителя
                if self.id_ not in c.children: # если айди инстанса не среди айди потомков выбранного родителя
                    c.children[self.id_] = 1 # записать его в потомки выбранного родителя
                else: # если айди уже среди айди потомков
                    c.children[self.id_] += 1 # добавить еще единичку (т.е. учесть, через какое количество веток 
                                                            # наш инстанс является наследником)
    
    def all_children_grads_accounted_for(self):
        """Проверяет, получил ли тензор градиенты от всех своих потомков в графе"""
        for id_, cnt in self.children.items(): # словарь self.children имеет ключи id_ и значения 1,2,...
            if cnt != 0: # если наследников нет, вернуть False
                return False
        return True # если наследник(и) есть, вернуть True
    
    def backward(self,grad=None, grad_origin=None):
        """Рекурсивно распространяет назад градиент
        от потомков к родителям """
        if(self.autograd): # если включатель автоградиента вкл. (если для нейрона нужно вычислять градиент)
#             if(grad is None): # если градиента пока никакого нет
#                 grad = FloatTensor(np.ones_like(self.data))
            
            if(grad_origin is not None): # если есть градиент
                if(self.children[grad_origin.id_] == 0): # и если потомков дальше нет
                    raise Exception("cannot backprop more than once") # не распространять градиент
                else: # если потомки есть
                    self.children[grad_origin.id_] -= 1 # скинуть счетчик потомков на 1

            if(self.grad is None): # если у инстанса нет градиента
                self.grad = grad # присвоить атрибут метода grad
            else: # если есть градиент
                self.grad += grad # добавить к нему значение атрибута метода grad
            
#             assert grad.autograd == False # проверка: если для градиента включен автоградиент, программу завалить
            
            # only continue backpropping if there's something to
            # backprop into and if all gradients (from children)
            # are accounted for override waiting for children if
            # "backprop" was called on this variable directly
            if(self.creators is not None and # если есть родители и одновременно есть наследники или grad_origin is None
               (self.all_children_grads_accounted_for() or 
                grad_origin is None)):

                if(self.creation_op == "add"): # если операция сложения
                    self.creators[0].backward(self.grad, self) # распространяем градиент на папу
                    self.creators[1].backward(self.grad, self) # и на маму
                if(self.creation_op == "neg"): # если операция сложения
                    self.creators[0].backward(self.grad.__neg__()) # распространяем градиент на папу
                if(self.creation_op == "sub"):
                    new = Tensor(self.grad.data)
                    self.creators[0].backward(new, self)
                    new = Tensor(self.grad.__neg__().data)
                    self.creators[1].backward(self)
                if(self.creation_op == "mul"):
                    new = self.grad * self.creators[1]
                    self.creators[0].backward(new, self)
                    new = self.grad * self.creators[0]
                    self.creators[1].backward(self)
                if(self.creation_op == "mm"):
                    act = self.creators[0]
                    weights = self.creators[1] # как правило слой активации
                    new = self.grad.mm(weights.transpose())
                    act.backward(new)
                    new = self.grad.transpose().mm(act).transpose()
                    weights.backward(new)
                if(self.creation_op == "transpose"): 
                    self.creators[0].backward(self.grad.transpose())
                if 'sum' in self.creation_op: 
                    dim = int(self.creation_op.split("_")[1])
                    ds = self.creators[0].data.shape[dim]
                    self.creators[0].backward(self.grad.expand(dim, ds))
                if 'expand' in self.creation_op: 
                    dim = int(self.creation_op.split("_")[1])
                    self.creators[0].backward(self.grad.sum(dim))
    
    def __add__(self, other):
        "Создает вычислительный граф"
        if self.autograd and other.autograd:
            
            return Tensor(self.data + other.data,
                            autograd=True,
                            creators = [self, other],
                            creation_op = "add")
        return Tensor(self.data + other.data)
    
    def __neg__(self):
        if self.autograd:
            return Tensor(self.data * (-1),
                 autograd = True, 
                 creators=[self], 
                 creation_op="neg")
        return Tensor(self.data * (-1))
    
    def __sub__(self, other):
        if self.autograd and other.autograd:
            return Tensor(self.data - other.data,
                            autograd=True,
                            creators = [self, other],
                            creation_op = "sub")
        return Tensor(self.data - other.data)
    
    def __mul__(self, other):
        if self.autograd and other.autograd:
            return Tensor(self.data * other.data,
                            autograd=True,
                            creators = [self, other],
                            creation_op = "mul")
        return Tensor(self.data * other.data)
    
    def __sum__(self, dim):
        """Выполняет сложение тензора по измерениям.
        При dim=0 сложение будет по столбцам, при dim=1 будут сложены строки"""
        if self.autograd:
            return Tensor(self.data.sum(dim),
                            autograd=True,
                            creators = [self],
                            creation_op = "sum_"+str(dim))
        return Tensor(self.data.sum(dim))
    
    def expand(self, dim, copies):
        """Используется для обратного распространения операции сложения.
        Копирует по измерению матрицу и размножает ее в количестве copies"""
        trans_cmd = list(range(0, len(data.shape))) # [0] для формы из одного измерения
        trans_cmd.insert(dim,  len(data.shape)) # вставляем в конец списка значение количества измерений, получаем [0, 1]
                                            # это нужно для параметра axes функции np.transpose()
        new_shape = list(data.shape) + [copies] # [2] для вектора из двух значений + [2] для двух копий,
                                            # получаем [2,2]
        new_data = data.repeat(copies).reshape(new_shape) # np.array([1,2]).repeat(2) возвращает array([1, 1, 2, 2])
                                                    # и решейпит на (2,2)
        new_data = new_data.transpose(trans_cmd) # решейпим функцией np.transpose()
        
        if self.autograd:
            return Tensor(new_data,
                            autograd=True,
                            creators = [self],
                            creation_op = "expand_" + str(dim))
        return Tensor(new_data)
    
    
    def transpose(self):
        if self.autograd:
            return Tensor(self.data.transpose(),
                            autograd=True,
                            creators = [self, x],
                            creation_op = "transpose")
        return Tensor(self.data.transpose())
    
    def mm(self, x):
        if self.autograd:
            return Tensor(self.data.dot(x.data),
                            autograd=True,
                            creators = [self, x],
                            creation_op = "mm")
        return Tensor(self.data.dot(x.data))
    
    
    def __repr__(self):
        return str(self.data.__repr__())
    
    def __str__(self):
        return str(self.data.__str__())

Пример сети, в которой мы вручную делаем обратное распространение

In [41]:
import numpy as np
np.random.seed(0)

data = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])  # (4,2)
target = np.array([[0],[1],[0],[1]]) # (4,1)

weights_0_1 = np.random.rand(2,3) # (2,3)
weights_1_2 = np.random.rand(3,1) #(3,1)

iterations = 10
alpha = 0.1

for i in range(iterations):
    layer_1 = data.dot(weights_0_1) # (4,2).dot(2,3) = (4,3)
    layer_2 = layer_1.dot(weights_1_2) # (4,3).dot(3,1) = (4,1)
    
    diff = layer_2 - target # (4,1) то же что и layer_2_grad
    sqdiff = diff ** 2 # (4,1)
    loss = np.sum(sqdiff) # скаляр
    
    layer_1_grad = diff.dot(weights_1_2.T) # (4,1).dot(1,3)=(4,3)
    weights_1_2_delta = layer_1.T.dot(diff) # (3,4).dot(4,1) = (3,1)
    weights_0_1_delta = data.T.dot(layer_1_grad) # (2,4).dot(4,3)=(2,3)
    
    weights_0_1 -= weights_0_1_delta * alpha
    weights_1_2 -= weights_1_2_delta * alpha
    
    print(loss)

5.066439994622395
0.4959907791902342
0.4180671892167177
0.35298133007809646
0.2972549636567377
0.2492326038163328
0.20785392075862477
0.17231260916265176
0.14193744536652986
0.11613979792168384


Больше не нужно писать вручную логику обратного распространения. Начинаем использовать наш фреймворк

In [43]:
import numpy as np

class Tensor (object):
    
    def __init__(self,data,
                 autograd=False,
                 creators=None,
                 creation_op=None,
                 id=None):
        
        self.data = np.array(data)
        self.autograd = autograd
        self.grad = None
        if(id is None):
            self.id = np.random.randint(0,100000)
        else:
            self.id = id
        
        self.creators = creators
        self.creation_op = creation_op
        self.children = {}
        
        if(creators is not None):
            for c in creators:
                if(self.id not in c.children):
                    c.children[self.id] = 1
                else:
                    c.children[self.id] += 1

    def all_children_grads_accounted_for(self):
        for id,cnt in self.children.items():
            if(cnt != 0):
                return False
        return True 
        
    def backward(self,grad=None, grad_origin=None):
        if(self.autograd):
 
            if(grad is None):
                grad = Tensor(np.ones_like(self.data))

            if(grad_origin is not None):
                if(self.children[grad_origin.id] == 0):
                    raise Exception("cannot backprop more than once")
                else:
                    self.children[grad_origin.id] -= 1

            if(self.grad is None):
                self.grad = grad
            else:
                self.grad += grad
            
            assert grad.autograd == False
            
            if(self.creators is not None and 
               (self.all_children_grads_accounted_for() or 
                grad_origin is None)):

                if(self.creation_op == "add"):
                    self.creators[0].backward(self.grad, self)
                    self.creators[1].backward(self.grad, self)
                    
                if(self.creation_op == "sub"):
                    self.creators[0].backward(Tensor(self.grad.data), self)
                    self.creators[1].backward(Tensor(self.grad.__neg__().data), self)

                if(self.creation_op == "mul"):
                    new = self.grad * self.creators[1]
                    self.creators[0].backward(new , self)
                    new = self.grad * self.creators[0]
                    self.creators[1].backward(new, self)                    
                    
                if(self.creation_op == "mm"):
                    c0 = self.creators[0]
                    c1 = self.creators[1]
                    new = self.grad.mm(c1.transpose())
                    c0.backward(new)
                    new = self.grad.transpose().mm(c0).transpose()
                    c1.backward(new)
                    
                if(self.creation_op == "transpose"):
                    self.creators[0].backward(self.grad.transpose())

                if("sum" in self.creation_op):
                    dim = int(self.creation_op.split("_")[1])
                    self.creators[0].backward(self.grad.expand(dim,
                                                               self.creators[0].data.shape[dim]))

                if("expand" in self.creation_op):
                    dim = int(self.creation_op.split("_")[1])
                    self.creators[0].backward(self.grad.sum(dim))
                    
                if(self.creation_op == "neg"):
                    self.creators[0].backward(self.grad.__neg__())
                    
    def __add__(self, other):
        if(self.autograd and other.autograd):
            return Tensor(self.data + other.data,
                          autograd=True,
                          creators=[self,other],
                          creation_op="add")
        return Tensor(self.data + other.data)

    def __neg__(self):
        if(self.autograd):
            return Tensor(self.data * -1,
                          autograd=True,
                          creators=[self],
                          creation_op="neg")
        return Tensor(self.data * -1)
    
    def __sub__(self, other):
        if(self.autograd and other.autograd):
            return Tensor(self.data - other.data,
                          autograd=True,
                          creators=[self,other],
                          creation_op="sub")
        return Tensor(self.data - other.data)
    
    def __mul__(self, other):
        if(self.autograd and other.autograd):
            return Tensor(self.data * other.data,
                          autograd=True,
                          creators=[self,other],
                          creation_op="mul")
        return Tensor(self.data * other.data)    

    def sum(self, dim):
        if(self.autograd):
            return Tensor(self.data.sum(dim),
                          autograd=True,
                          creators=[self],
                          creation_op="sum_"+str(dim))
        return Tensor(self.data.sum(dim))
    
    def expand(self, dim,copies):

        trans_cmd = list(range(0,len(self.data.shape)))
        trans_cmd.insert(dim,len(self.data.shape))
        new_data = self.data.repeat(copies).reshape(list(self.data.shape) + [copies]).transpose(trans_cmd)
        
        if(self.autograd):
            return Tensor(new_data,
                          autograd=True,
                          creators=[self],
                          creation_op="expand_"+str(dim))
        return Tensor(new_data)
    
    def transpose(self):
        if(self.autograd):
            return Tensor(self.data.transpose(),
                          autograd=True,
                          creators=[self],
                          creation_op="transpose")
        
        return Tensor(self.data.transpose())
    
    def mm(self, x):
        if(self.autograd):
            return Tensor(self.data.dot(x.data),
                          autograd=True,
                          creators=[self,x],
                          creation_op="mm")
        return Tensor(self.data.dot(x.data))
    
    def __repr__(self):
        return str(self.data.__repr__())
    
    def __str__(self):
        return str(self.data.__str__())  

In [46]:
import numpy as np
np.random.seed(0)

data = Tensor(np.array([[0, 0], [0, 1], [1, 0], [1, 1]]), autograd=True)  # (4,2)
target = Tensor(np.array([[0],[1],[0],[1]]), autograd=True) # (4,1)

w = []
w.append(Tensor(np.random.rand(2,3), autograd=True)) # (2,3)
w.append(Tensor(np.random.rand(3,1), autograd=True)) #(3,1)

iterations = 10
alpha = 0.1

for i in range(iterations):
    pred = data.mm(w[0]).mm(w[1]) # (4,1)
    loss = ((pred - target)*(pred - target)).sum(0) # скаляр
    loss.backward(Tensor(np.ones_like(loss.data)))
    
    for w_ in w:
        w_.data -= w_.grad.data * alpha
        w_.grad.data *= 0
    
    print(loss)

[0.58128304]
[0.48988149]
[0.41375111]
[0.34489412]
[0.28210124]
[0.2254484]
[0.17538853]
[0.1324231]
[0.09682769]
[0.06849361]


Добавим SGD

In [52]:
class SGD(object):
    
    def __init__(self, params, alpha=0.1):
        self.params = params
        self.alpha = alpha
        
    def zero(self):
        for p in self.params:
            p.data -= p.grad.data * self.alpha
            if zero:
                p.grad.data *= 0     
                
    def step(self, zero=True):
        for p in self.params:
            p.data -= p.grad.data * self.alpha
            
            if zero:
                p.grad.data *= 0

In [53]:
import numpy as np
np.random.seed(0)

data = Tensor(np.array([[0, 0], [0, 1], [1, 0], [1, 1]]), autograd=True)  # (4,2)
target = Tensor(np.array([[0],[1],[0],[1]]), autograd=True) # (4,1)

w = []
w.append(Tensor(np.random.rand(2,3), autograd=True)) # (2,3)
w.append(Tensor(np.random.rand(3,1), autograd=True)) #(3,1)

iterations = 10
optim = SGD(params=w, alpha=0.1)

for i in range(iterations):
    pred = data.mm(w[0]).mm(w[1]) # (4,1)
    loss = ((pred - target)*(pred - target)).sum(0) # скаляр
    loss.backward(Tensor(np.ones_like(loss.data)))
    
    optim.step()
    
    print(loss)

[0.58128304]
[0.48988149]
[0.41375111]
[0.34489412]
[0.28210124]
[0.2254484]
[0.17538853]
[0.1324231]
[0.09682769]
[0.06849361]


Добавление новых слоев

In [70]:
class Layer():
    
    def __init__(self):
        self.params = []
    
    def get_params(self):
        return self.params

class Linear(Layer):
    
    def __init__(self, n_inputs, n_outputs):
        super().__init__()
        # np.random.randn возвращает массив нормально распределенных величин около 0
        # размера (n_inputs, n_outputs)
        W = np.random.randn(n_inputs, n_outputs) * np.sqrt(2.0/n_inputs)
        self.weight = Tensor(W, autograd=True)
        self.bias = Tensor(np.zeros(n_outputs), autograd=True)
        
        self.params.append(self.weight)
        self.params.append(self.bias)
        
    def forward(self, inp):
        return inp.mm(self.weight)+self.bias.expand(0, len(inp.data))

Sequential слои

In [71]:
class Sequential(Layer):
    
    def __init__(self, layers=list()):
        super().__init__()
        
        self.layers = layers
        
    def add(self, layer):
        self.layers.append(layer)
        
    def forward(self, inp):
        for layer in self.layers:
            inp = layer.forward(inp)
        return inp
    
    def get_params(self):
        params = []
        for l in self.layers:
            params += l.get_params()
        return params

In [73]:
import numpy as np
np.random.seed(0)

data = Tensor(np.array([[0, 0], [0, 1], [1, 0], [1, 1]]), autograd=True)  # (4,2)
target = Tensor(np.array([[0],[1],[0],[1]]), autograd=True) # (4,1)
l_list = [Linear(2,3), Linear(3,1)]

model = Sequential(l_list)
optim = SGD(params=model.get_params(), alpha = 0.05)
iterations = 10

for i in range(iterations):
    pred = model.forward(data)
    loss = ((pred - target)*(pred - target)).sum(0)
    loss.backward(Tensor(np.ones_like(loss.data)))
    optim.step()
    print(loss)

[2.33428272]
[0.06743796]
[0.0521849]
[0.04079507]
[0.03184365]
[0.02479336]
[0.01925443]
[0.01491699]
[0.01153118]
[0.00889602]


Слои с функцией потерь

In [74]:
class MSELoss(Layer):
    
    def __init__(self):
        super().__init__()
        
    def forward(self, pred, target):
        return ((pred - target)*(pred - target)).sum(0)

In [75]:
import numpy as np
np.random.seed(0)

data = Tensor(np.array([[0, 0], [0, 1], [1, 0], [1, 1]]), autograd=True)  # (4,2)
target = Tensor(np.array([[0],[1],[0],[1]]), autograd=True) # (4,1)
l_list = [Linear(2,3), Linear(3,1)]

model = Sequential(l_list)
optim = SGD(params=model.get_params(), alpha = 0.05)
iterations = 10
criterion = MSELoss()

for i in range(iterations):
    pred = model.forward(data)
    loss = criterion.forward(pred, target)
    loss.backward(Tensor(np.ones_like(loss.data)))
    optim.step()
    print(loss)

[2.33428272]
[0.06743796]
[0.0521849]
[0.04079507]
[0.03184365]
[0.02479336]
[0.01925443]
[0.01491699]
[0.01153118]
[0.00889602]
